In [8]:
import pandas as pd


In [61]:
# ============================================
# CÉLULA 1 — EXTRACT (ler a base de dados)
# ============================================

# Definição dos caminhos
file_name = "nintendo_packages"

# Monta o caminho completo adicionando a extensão .xlsx
entry_path = f"../Python/data/raw/{file_name}.xlsx"
output_path = f"../Python/data/processed/{file_name}_processed.xlsx"

try:
    df = pd.read_excel(entry_path)
    print("✅ Arquivo carregado com sucesso!")
    print("Dimensões do dataset:", df.shape)
    print("Colunas disponíveis:", df.columns.tolist())
    
    # Visualização dos dados
    print("\nDados carregados:")

    # Visualização de todos os dados
    #pd.set_option("display.max_rows", None)
    #print(df)

    # Visualização das 5 primeiras linhas
    print(df.head()) # mostra as 5 primeiras linhas
    
except FileNotFoundError:
    print("❌ Arquivo não encontrado. Verifique o caminho.")
except Exception as e:
    print("❌ Erro ao carregar o arquivo:", e)


✅ Arquivo carregado com sucesso!
Dimensões do dataset: (118, 12)
Colunas disponíveis: ['Package Id', 'Game', 'Send Date', 'Region', 'Order Status', 'E-commerce', 'Stock Quantity', 'Preço Unit Price', 'Store Manager', 'Batch', 'Age Rating', 'Publisher']

Dados carregados:
  Package Id                                              Game  \
0      95774  The Legend of Zelda: Breath of the Wild - Switch   
1      44399  The Legend of Zelda: Breath of the Wild - Switch   
2      33068                            Pokémon Sword - Switch   
3      44752                            Pokémon Sword - Switch   
4      90789                           Pokémon Shield - Switch   

             Send Date Region Order Status    E-commerce Stock Quantity  \
0  2024-04-15 00:00:00     EN      Pending      GameShop            167   
1  2023-06-04 00:00:00     PT      Pending      GameShop             91   
2  2023-09-02 00:00:00     JP      Pending     PlayWorld             50   
3  2024-09-07 00:00:00     JP  

In [239]:
# ========================================================================================#

In [240]:
# ------------------------------------------------------
# CÉLULA 2 — TRANSFORM (manipulação dos dados)
# ------------------------------------------------------

In [62]:
# 2.1. Excluir a coluna 'Package Id'

if "Package Id" in df.columns:
    df = df.drop("Package Id", axis=1)
    
    # Visualização das 5 primeiras linhas
    print("✅ Coluna 'Package Id' removida com sucesso!\n")
    print(df.head())
else:
    print("⚠️ Coluna 'Package Id' não encontrada no dataset.")

✅ Coluna 'Package Id' removida com sucesso!

                                               Game            Send Date  \
0  The Legend of Zelda: Breath of the Wild - Switch  2024-04-15 00:00:00   
1  The Legend of Zelda: Breath of the Wild - Switch  2023-06-04 00:00:00   
2                            Pokémon Sword - Switch  2023-09-02 00:00:00   
3                            Pokémon Sword - Switch  2024-09-07 00:00:00   
4                           Pokémon Shield - Switch  2023-04-09 00:00:00   

  Region Order Status    E-commerce Stock Quantity Preço Unit Price  \
0     EN      Pending      GameShop            167            50.07   
1     PT      Pending      GameShop             91            49.16   
2     JP      Pending     PlayWorld             50            57.54   
3     JP    Completed      GameShop            177            52.33   
4     JP    Completed  DigitalStore            169            53.19   

    Store Manager      Batch Age Rating    Publisher  
0  Crystal Hayne

In [63]:
# 2.2. Limpeza de valores inválidos e remoção de linhas incompletas

try:
    # Lista de valores inválidos
    invalid_values = ["---", "----", "-----"]

    # Substitui todos os valores inválidos por NaN
    df = df.replace(invalid_values, pd.NA)

    # Agora remove as linhas que ficaram com NaN
    linhas_antes = df.shape[0]
    df = df.dropna(how="any")
    linhas_depois = df.shape[0]

    # Visualizar a prévia
    print(f"✅ Limpeza concluída! ({linhas_antes - linhas_depois} linhas removidas.)")
    print("Dimensões após limpeza:", df.shape)

    pd.set_option("display.max_rows", None)

    print("\nPrévia dos dados limpos:")
    print(df.head())
except Exception as e:
    print("❌ Erro na limpeza dos dados:", e)

✅ Limpeza concluída! (9 linhas removidas.)
Dimensões após limpeza: (109, 11)

Prévia dos dados limpos:
                                               Game            Send Date  \
0  The Legend of Zelda: Breath of the Wild - Switch  2024-04-15 00:00:00   
1  The Legend of Zelda: Breath of the Wild - Switch  2023-06-04 00:00:00   
2                            Pokémon Sword - Switch  2023-09-02 00:00:00   
3                            Pokémon Sword - Switch  2024-09-07 00:00:00   
4                           Pokémon Shield - Switch  2023-04-09 00:00:00   

  Region Order Status    E-commerce Stock Quantity Preço Unit Price  \
0     EN      Pending      GameShop            167            50.07   
1     PT      Pending      GameShop             91            49.16   
2     JP      Pending     PlayWorld             50            57.54   
3     JP    Completed      GameShop            177            52.33   
4     JP    Completed  DigitalStore            169            53.19   

    Store Man

In [64]:
# 2.3. Criar a coluna "Game Console"

try:
    # Verifica se a coluna existe
    if "Game" in df.columns:
        # Criar coluna "Game Console" (parte depois do "-")
        df["Game Console"] = df["Game"].str.split("-").str[-1].str.strip()

        # Visualizar prévia
        print("✅ Coluna 'Game Console' criada com sucesso!\n")
        print(df[["Game","Game Console"]].head())
        
    else:
        print("⚠️ Coluna 'Game' não encontrada no dataset.")
except Exception as e:
    print("❌ Erro ao criar a coluna 'Game Console':", e)

✅ Coluna 'Game Console' criada com sucesso!

                                               Game Game Console
0  The Legend of Zelda: Breath of the Wild - Switch       Switch
1  The Legend of Zelda: Breath of the Wild - Switch       Switch
2                            Pokémon Sword - Switch       Switch
3                            Pokémon Sword - Switch       Switch
4                           Pokémon Shield - Switch       Switch


In [65]:
 # 2.4. Ajustar as colunas "Game" e "Batch"

try:
    if "Game" in df.columns:
        # Ajustar coluna "Game" (parte antes do "-")
        df["Game"] = df["Game"].str.split("-").str[0].str.strip()

    if "Batch" in df.columns:
        # Ajustar coluna "Batch" (parte antes do "-")
        df["Batch"] = df["Batch"].str.split("-").str[-1].str.strip()

    print("✅ Colunas 'Game' e 'Batch' ajustadas com sucesso!\n")
    print(df[["Game", "Batch"]].head())
except Exception as e:
    print("❌ Erro ao ajustar colunas 'Game' e 'Batch':", e)

✅ Colunas 'Game' e 'Batch' ajustadas com sucesso!

                                      Game Batch
0  The Legend of Zelda: Breath of the Wild   202
1  The Legend of Zelda: Breath of the Wild   202
2                            Pokémon Sword   047
3                            Pokémon Sword   047
4                           Pokémon Shield   115


In [66]:
# 2.5. Ajustar a coluna Send Date

try:
    if "Send Date" in df.columns:
        # Converter para datetime e formatar no padrão brasileiro
        df["Send Date"] = pd.to_datetime(df["Send Date"], errors="coerce").dt.strftime("%d/%m/%Y")
                
        # Visualizar prévia
        print("✅ Coluna 'Send Date' convertida para formato brasileiro (DD/MM/YYYY) com sucesso!\n")
        print(df[["Send Date"]].head())
    else:
        print("⚠️ Coluna 'Send Date' não encontrada no dataset.")
except Exception as e:
    print("❌ Erro ao ajustar a coluna 'Send Date':", e)


✅ Coluna 'Send Date' convertida para formato brasileiro (DD/MM/YYYY) com sucesso!

    Send Date
0  15/04/2024
1  04/06/2023
2  02/09/2023
3  07/09/2024
4  09/04/2023


In [67]:
# 2.6. Ajustar a coluna "Region"

try:
    if "Region" in df.columns:
        # Mapeamento das siglas para nomes completos
        region_map = {
            "EN": "English",
            "PT": "Portuguese",
            "JP": "Japanese"
        }

        # Substituir os valores
        df["Region"] = df["Region"].replace(region_map)
        
         # Visualizar prévia
        print("✅ Coluna 'Region' ajustada com sucesso!\n")
        print(df[["Region"]].head())
    else:
        print("⚠️ Coluna 'Region' não encontrada no dataset.")
except Exception as e:
    print("❌ Erro ao ajustar a coluna 'Region':", e)


✅ Coluna 'Region' ajustada com sucesso!

       Region
0     English
1  Portuguese
2    Japanese
3    Japanese
4    Japanese


In [68]:
# 2.7. Renomear e ajustar a coluna "Preço Unit Price" para "Unit Price"

import re

try:
    if "Preço Unit Price" in df.columns:
        # 2.7.1. Renomear a coluna
        df = df.rename(columns={"Preço Unit Price": "Unit Price"})
        print("✅ Coluna 'Preço Unit Price' renomeada para 'Unit Price'.")

        # 2.7.2. Função robusta para normalizar string de preço
        def normalize_price(val):
            if pd.isna(val):
                return pd.NA
            s = str(val).strip()

            # Remover tudo que não seja dígito, vírgula, ponto
            s = re.sub(r"[^0-9.,]", "", s)

            # Casos:
            # a) "1.234,56" (pt-BR): ponto milhar, vírgula decimal → remove pontos, vírgula -> ponto
            if "," in s and "." in s:
                # Se a vírgula estiver à direita do último ponto, tratamos como pt-BR
                # Ex: 1.234,56 → remove ".", troca "," por "."
                s = s.replace(".", "").replace(",", ".")
            # b) "50,07" (pt-BR simples): vírgula decimal → troca vírgula por ponto
            elif "," in s and "." not in s:
                s = s.replace(",", ".")
            # c) "1,234.56" (en-US): vírgula milhar, ponto decimal → remove vírgulas
            elif "." in s and "," in s:
                s = s.replace(",", "")

            # d) "1234.56" ou "50" já ok

            try:
                return float(s)
            except:
                return pd.NA  

        # 2.7.3. Aplicar normalização
        df["Unit Price"] = df["Unit Price"].apply(normalize_price)

        # 2.7.4. Feedback de conversão
        na_count = df["Unit Price"].isna().sum()
        print("✅ Coluna 'Unit Price' normalizada e convertida para numérico\n")

        print(df[["Unit Price"]].head())

    else:
        print("⚠️ Coluna 'Preço Unit Price' não encontrada no dataset.")
except Exception as e:
    print("❌ Erro ao ajustar a coluna 'Unit Price':", e)



✅ Coluna 'Preço Unit Price' renomeada para 'Unit Price'.
✅ Coluna 'Unit Price' normalizada e convertida para numérico

   Unit Price
0       50.07
1       49.16
2       57.54
3       52.33
4       53.19


In [69]:
# 2.8. Criar a coluna "Total Price"

try:
    if "Stock Quantity" in df.columns and "Unit Price" in df.columns:
        # Multiplicar quantidade pelo preço unitário
        df["Total Price"] = df["Stock Quantity"].astype(float) * df["Unit Price"].astype(float)

        # Visualizar prévia
        print("✅ Coluna 'Total Price' criada com sucesso!\n")
        print(df[["Stock Quantity", "Unit Price", "Total Price"]].head())
    else:
        print("⚠️ Colunas necessárias ('Stock Quantity' e/ou 'Unit Price') não encontradas no dataset.")
except Exception as e:
    print("❌ Erro ao criar a coluna 'Total Price':", e)


✅ Coluna 'Total Price' criada com sucesso!

  Stock Quantity  Unit Price  Total Price
0            167       50.07      8361.69
1             91       49.16      4473.56
2             50       57.54      2877.00
3            177       52.33      9262.41
4            169       53.19      8989.11


In [72]:
# ---------------------------
# CÉLULA 3 — LOAD (salvar o resultado)
# ---------------------------

# Caminho de saída já definido anteriormente:
output_path = f"../Python/data/processed/{file_name}_processed.xlsx"

try:
    df.to_excel(output_path, index=False)
    print(f"✅ Dataset processado salvo com sucesso em: {output_path}")
except Exception as e:
    print("❌ Erro ao salvar o arquivo:", e)


✅ Dataset processado salvo com sucesso em: ../Python/data/processed/nintendo_packages_processed.xlsx
